In [14]:
import sys, Quandl,nltk, csv, urllib2, codecs, simplejson, decimal, string, json
import numpy as np
from pprint import pprint as pp
from nltk.corpus import movie_reviews
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
from pymongo import MongoClient
import pandas as pd
from operator import itemgetter
import matplotlib.pyplot as plt
import json
from IPython.display import display_pretty, display_html, display_jpeg, display_png, display_json, display_latex, display_svg
%matplotlib inline

In [15]:
client = MongoClient()
db = client.dsbc
# nw = db.neuro_words
# nwfeq = db.nwfeq
# nwfeq2 = db.nwfeq2
nwf = db.nwf

# nw.delete_many({"$where":"this.content.length < 100"})
# nw.delete_many({"$where":"typeof(this.content) != 'string'"})

# contents = nw.find({},{"content":1, "_id":0})
# content_vals = [i['content'] for i in contents]

# len(content_vals)
with open('result.json') as data_file:    
    data = json.load(data_file)

# for rev in reviews:
#     hmm.save(rev)


In [16]:
# nwfeq.remove({})
# nwfeq2.remove({})
nwf.remove({})

for key, val in data.items():
#     if index == 10:
#         break
    bregion = key.split(" | ")[1]
    scale = key.split(" | ")[0]
    my_hash = {"bregion": bregion, "scale": scale, "co-occ_count": val}
    nwf.save(my_hash)

In [22]:
# cursor = nwfeq2.find({'scale': {"$ne" : "quest"}}).sort([("co-occ_count", -1)])
# cursor = nwf.find({"bregion":{"$ne" : ""}}, {"scale":{"$ne" : ""}}).sort([("co-occ_count", -1)])
# cursor = nwf.find({ "$and": [ { "scale": { "$ne": "" } }, { "bregion": { "$ne": "" } } ] }).sort([("co-occ_count", -1)])
cursor = nwf.find({ "$and": [ { "$where": "this.scale.length > 5" }, { "$where": "this.bregion.length > 5" } ] }).sort([("co-occ_count", -1)])
list(cursor)

[{u'_id': ObjectId('556ce8e1ea638733b700008f'),
  u'bregion': u'frontal cortex',
  u'co-occ_count': 94,
  u'scale': u'iowa gambling task'},
 {u'_id': ObjectId('556ce8e0ea638733b7ffe9ca'),
  u'bregion': u'frontal cortex',
  u'co-occ_count': 88,
  u'scale': u'reinforcement learning'},
 {u'_id': ObjectId('556ce8e0ea638733b7ffee0f'),
  u'bregion': u'frontal cortex',
  u'co-occ_count': 79,
  u'scale': u'wisconsin card sorting test'},
 {u'_id': ObjectId('556ce8dfea638733b7ffdd92'),
  u'bregion': u'prefrontal cortex',
  u'co-occ_count': 65,
  u'scale': u'iowa gambling task'},
 {u'_id': ObjectId('556ce8e2ea638733b70011f9'),
  u'bregion': u'prefrontal cortex',
  u'co-occ_count': 65,
  u'scale': u'wisconsin card sorting test'},
 {u'_id': ObjectId('556ce8e0ea638733b7ffe2cc'),
  u'bregion': u'prefrontal cortex',
  u'co-occ_count': 58,
  u'scale': u'reinforcement learning'},
 {u'_id': ObjectId('556ce8e0ea638733b7ffe4e0'),
  u'bregion': u'frontal lobe',
  u'co-occ_count': 50,
  u'scale': u'wisconsin

In [21]:
# cursor = nwfeq2.find({'scale': {"$ne" : "quest"}}).sort([("co-occ_count", -1)])
# cursor = nwf.find({"bregion":{"$ne" : ""}}, {"scale":{"$ne" : ""}}).sort([("co-occ_count", -1)])
# cursor = nwf.find({ "$and": [ { "scale": { "$ne": "" } }, { "bregion": { "$ne": "" } } ] }).sort([("co-occ_count", -1)])
cursor = nwf.find({ "$and": [ { "$where": "this.scale.length > 5" }, { "$where": "this.bregion.length > 5" } ] }).sort([("co-occ_count", -1)])

list(cursor)

[{u'_id': ObjectId('556ce8e1ea638733b700008f'),
  u'bregion': u'frontal cortex',
  u'co-occ_count': 94,
  u'scale': u'iowa gambling task'},
 {u'_id': ObjectId('556ce8e0ea638733b7ffe9ca'),
  u'bregion': u'frontal cortex',
  u'co-occ_count': 88,
  u'scale': u'reinforcement learning'},
 {u'_id': ObjectId('556ce8e0ea638733b7ffee0f'),
  u'bregion': u'frontal cortex',
  u'co-occ_count': 79,
  u'scale': u'wisconsin card sorting test'},
 {u'_id': ObjectId('556ce8dfea638733b7ffdd92'),
  u'bregion': u'prefrontal cortex',
  u'co-occ_count': 65,
  u'scale': u'iowa gambling task'},
 {u'_id': ObjectId('556ce8e2ea638733b70011f9'),
  u'bregion': u'prefrontal cortex',
  u'co-occ_count': 65,
  u'scale': u'wisconsin card sorting test'},
 {u'_id': ObjectId('556ce8e0ea638733b7ffe2cc'),
  u'bregion': u'prefrontal cortex',
  u'co-occ_count': 58,
  u'scale': u'reinforcement learning'},
 {u'_id': ObjectId('556ce8e0ea638733b7ffe4e0'),
  u'bregion': u'frontal lobe',
  u'co-occ_count': 50,
  u'scale': u'wisconsin

In [11]:
# cursor_grouped = nwfeq2.aggregate(
#    [
#      { "$group" : { "_id" : "$co-occur_count", "nwfeq2": { "$push": "$scale" } } }
#    ]
# )
# list(cursor_grouped)

In [3]:
# vectorizer = TfidfVectorizer(stop_words="english", lowercase=False, encoding=u'utf-8')
# doc_vectors = vectorizer.fit_transform(content_vals)
# idf = vectorizer._tfidf.idf_
# idf_dic = dict(zip(vectorizer.get_feature_names(), idf))
# def format_brain(fname):
#     my_file = open(fname, "r")
#     read_file = my_file.read()
#     my_file.close()
#     return list(set(read_file.translate(string.maketrans("",""), string.punctuation).lower().split("\n")))

# def format_scale(fname):
#     my_file = open(fname, "r")
#     lines = list(set(my_file.read().split("\r")))
#     return [line.split(',')[0] for line in lines]

# bterms = format_brain("mesh_brain3.txt")
# sterms = format_scale("mmy_index.txt")

In [12]:
# nw2 = db.neuro_words2
# nw2.delete_many({"$where":"this.content.length < 100"})
# nw2.delete_many({"$where":"typeof(this.content) != 'string'"})
# thing = nw.find({},{"scales":1, "_id":0})
# list(thing)